# DONT RUN THIS AGAIN

In [10]:
import numpy as np
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from llm_calc.lib.experiment import  ls_client, make_df
from llm_calc.lib.config import config
from llm_calc.lib.datamodel import Arm, ArmSlug, Model, ModelSlug
from llm_calc.lib.datacore import datacore
from os.path import join as path_join
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

In [12]:
queue_names = [ "LLMCalc -- Error Classification Test Queue",
                "LLMCalc -- Error Classification Alex Goodell",
                "LLMCalc -- Error Classification Simon Chu",
                "LLMCalc -- Error Classification Larry Chu",
           ]

In [13]:
dataset_queues_df = pd.DataFrame()

for queue_name in queue_names:
    dataset_queues = pd.DataFrame(
        ls_client.list_annotation_queues(name=queue_name)
    )
    if len(dataset_queues) > 0:
         qdf = make_df(dataset_queues)
         # qid = qdf.id.item()
         # an = ls_client.read_annotation_queue(queue_id=qid) # doesn't give you any more data
         # print(an)
         # andf = make_df(an)
         dataset_queues_df = pd.concat([dataset_queues_df, qdf])

dataset_queues_df

,id,name,description,created_at,updated_at,tenant_id
0,411ece5f-98b1-4211-81b8-5c2a7dee0aa6,LLMCalc -- Error Classification Alex Goodell,annotation queue for LLMCalc -- Error Classification Alex Goodell,2024-11-01 19:52:47.021591,2024-11-15 16:22:03.140289,80231dbb-1e31-4379-b804-df697b777bc6
0,e7de1584-c983-4f33-86f3-70eca8169028,LLMCalc -- Error Classification Simon Chu,annotation queue for LLMCalc -- Error Classification Simon Chu,2024-11-01 19:52:48.095032,2024-11-01 19:52:48.478521,80231dbb-1e31-4379-b804-df697b777bc6
0,e949624e-037c-44b3-9a3c-401e8cbdfc9a,LLMCalc -- Error Classification Larry Chu,annotation queue for LLMCalc -- Error Classification Larry Chu,2024-11-01 19:52:48.902326,2024-11-04 23:56:50.050865,80231dbb-1e31-4379-b804-df697b777bc6


In [14]:
def get_runs_from_q(qname, num):
    runs = pd.DataFrame()
    qid = dataset_queues_df[dataset_queues_df.name == qname].iloc[0].id
    for indx in range(0,num):
        rn = ls_client.get_run_from_annotation_queue(queue_id=qid, index=indx)
        run_id = rn.id
        single_run_df = make_df(ls_client.list_runs(run_ids=[run_id]))
        runs = pd.concat([runs, single_run_df])
    return runs

In [15]:
runs = get_runs_from_q("LLMCalc -- Error Classification Alex Goodell", 2)

In [16]:
runs.iloc[0].id

UUID('211e216d-dd21-4dd8-b482-db14d90f17b4')

In [17]:
runs["assigned_to"] = "hi"

In [18]:
# these numbers need to be manually updated from the UI prior to running this script
queue_names_and_sizes = [   ("LLMCalc -- Error Classification Alex Goodell", 125),
                            ("LLMCalc -- Error Classification Simon Chu", 150),
                            ("LLMCalc -- Error Classification Larry Chu", 90) ]

queued_runs = pd.DataFrame()
for queue_name_and_size in queue_names_and_sizes:
    resp_df = get_runs_from_q(queue_name_and_size[0],queue_name_and_size[1])
    person = queue_name_and_size[0].replace('LLMCalc -- Error Classification ','')
    resp_df["assigned_to"] = person
    filename = path_join(config.RESULTS_DATA_PATH, "annotation", f"subset {person}.pkl")
    resp_df.to_pickle(filename)
    queued_runs = pd.concat([queued_runs, resp_df])



queued_runs


,id,name,start_time,run_type,end_time,extra,error,serialized,events,inputs,...,total_tokens,first_token_time,total_cost,prompt_cost,completion_cost,parent_run_ids,trace_id,dotted_order,in_dataset,assigned_to
0,211e216d-dd21-4dd8-b482-db14d90f17b4,execute,2024-10-31 23:26:33.631384,chain,2024-10-31 23:26:46.397041,"{'metadata': {'revision_id': '69fd87a-dirty', 'arm': 'llama_omc', ...",None,None,[],{'inputs': {'human_input': 'I would like you assistance with a cas...,...,20651,2024-10-31 23:26:34.113937,0.0082604,0.0081368,0.0001236,[],211e216d-dd21-4dd8-b482-db14d90f17b4,20241031T232633631384Z211e216d-dd21-4dd8-b482-db14d90f17b4,False,Alex Goodell
0,766ad387-4b88-420e-88da-4abb68c12d10,execute,2024-11-01 18:24:34.530915,chain,2024-11-01 18:24:42.984254,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_rag_ci'...",None,None,[],{'inputs': {'human_input': 'I have a patient that I'd like to tell...,...,2884,2024-11-01 18:24:35.564089,0.01498,0.01414,0.00084,[],766ad387-4b88-420e-88da-4abb68c12d10,20241101T182434530915Z766ad387-4b88-420e-88da-4abb68c12d10,False,Alex Goodell
0,394c3306-1811-477c-8e57-27ad199c6d21,execute,2024-11-01 16:22:59.168627,chain,2024-11-01 16:23:09.437920,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_ci', 'e...",None,None,[],{'inputs': {'human_input': 'A 65-year-old female patient is admitt...,...,1434,2024-11-01 16:23:00.705941,0.00784,0.006835,0.001005,[],394c3306-1811-477c-8e57-27ad199c6d21,20241101T162259168627Z394c3306-1811-477c-8e57-27ad199c6d21,False,Alex Goodell
0,db70c639-20db-41b5-85b7-ac88f427295a,execute,2024-10-30 15:55:59.468902,chain,2024-10-30 15:56:15.859321,"{'metadata': {'revision_id': 'd3184dd-dirty', 'arm': 'gpt4_rag', '...",None,None,[],{'inputs': {'human_input': 'I'm seeking your advice on a liver tra...,...,2111,2024-10-30 15:56:00.945986,0.017225,0.00722,0.010005,[],db70c639-20db-41b5-85b7-ac88f427295a,20241030T155559468902Zdb70c639-20db-41b5-85b7-ac88f427295a,False,Alex Goodell
0,e931862e-d54f-444c-8e5e-20946c677a48,execute,2024-11-01 16:20:40.983930,chain,2024-11-01 16:20:45.895698,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_base', ...",None,None,[],{'inputs': {'human_input': 'Please assist with assessing a patient...,...,615,2024-11-01 16:20:41.486199,0.004385,0.00242,0.001965,[],e931862e-d54f-444c-8e5e-20946c677a48,20241101T162040983930Ze931862e-d54f-444c-8e5e-20946c677a48,False,Alex Goodell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,8c4a333c-f017-46ad-8e4f-ade06a8cd62b,execute,2024-10-30 10:05:12.185865,chain,2024-10-30 10:05:25.408894,"{'metadata': {'revision_id': '324c698-dirty', 'arm': 'llama_rag', ...",None,None,[],{'inputs': {'human_input': 'I have a patient that I would like you...,...,2109,2024-10-30 10:05:13.004543,0.0008436,0.0007732,0.0000704,[],8c4a333c-f017-46ad-8e4f-ade06a8cd62b,20241030T100512185865Z8c4a333c-f017-46ad-8e4f-ade06a8cd62b,False,Larry Chu
0,951bcf67-c506-4ebb-a54d-2b23f74d7717,execute,2024-11-01 02:59:28.752314,chain,2024-11-01 02:59:51.233586,"{'metadata': {'revision_id': 'c9d53ee-dirty', 'arm': 'llama_base',...",None,None,[],{'inputs': {'human_input': 'An 82-year-old man presents with new n...,...,6047,2024-11-01 02:59:30.016601,0.0024188,0.0023568,0.000062,[],951bcf67-c506-4ebb-a54d-2b23f74d7717,20241101T025928752314Z951bcf67-c506-4ebb-a54d-2b23f74d7717,False,Larry Chu
0,7e9fd2a9-6a57-41d7-b026-cd469793caef,execute,2024-11-01 05:50:42.269527,chain,2024-11-01 05:50:53.068571,"{'metadata': {'revision_id': '072afce-dirty', 'arm': 'llama_rag_ci...",None,None,[],{'inputs': {'human_input': 'I'd like your assistance with a patien...,...,4091,2024-11-01 05:50:43.888401,0.0016364,0.0015428,0.0000936,[],7e9fd2a9-6a57-41d7-b026-cd469793caef,20241101T055042269527Z7e9fd2a9-6a57-41d7-b026-cd469793caef,False,Larry Chu
0,b62e9e0d-8238-493f-b810-52d2d90bb520,execute,2024-11-01 03:15:05.082427,chain,2024-11-01 03:15:22.103409,"{'metadata': {'revision_id': 'c9d53ee-dirty', 'arm': 'll

### Get runs that have already been annotated

In [19]:
filename = path_join(config.RESULTS_DATA_PATH,"dataset_results_most-recent.pkl")
df = pd.read_pickle(filename)

In [20]:
# exclude last runs (not finished before annotation queue was made)
print(df.shape)
to_exclude = ["melted-cage", "efficacious-romaine", "successful-angina", "known-increase", "pastoral-constitution"]
df = df[~df.experiment_name.isin(to_exclude)]
print(df.shape)

(10000, 61)
(7500, 61)


In [21]:
df[499:500].id.to_list()

[UUID('cfda568d-d6f9-41f5-b759-265012a3873c')]

In [22]:
annotated_df = pd.DataFrame()
start = 0
end = 500
for i in range(20):
    print(start, end, end="...")
    foo = ls_client.list_runs(
        run_ids=df[start:end].id.to_list(),
        trace_filter='eq(feedback_key, "error classification")',
    )
    fdf = pd.DataFrame()
    try:
        fdf = make_df(foo)
    except:
        pass
    annotated_df = pd.concat([annotated_df, fdf])
    start = start + 500
    end = end + 500

annotated_df.shape

0 500...500 1000...1000 1500...1500 2000...2000 2500...2500 3000...3000 3500...3500 4000...4000 4500...4500 5000...5000 5500...5500 6000...6000 6500...6500 7000...7000 7500...7500 8000...8000 8500...8500 9000...9000 9500...9500 10000...

(49, 33)

In [23]:
# drop testing annotations (did not realize that they were all pointers and the testing annotation queue is real data)
revised_annotated_df = annotated_df.reset_index().drop([0, 7, 8, 9])

### Make HTML document with links to annotated runs

In [24]:
# used to check status of annotated runs
urls = list()
for i, row in revised_annotated_df.iterrows():
    new_row = { 'url': 'https://smith.langchain.com' + row['app_path'] }
    urls = urls + [new_row]

url_df = pd.DataFrame.from_records(urls)
filename = path_join(config.RESULTS_DATA_PATH, "annotation", f"annotated_runs.pkl")
revised_annotated_df.to_pickle(filename)
# Convert URLs to clickable HTML links
url_df['url'] = url_df['url'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')

# Set the 'url' column as the index to avoid duplicating it in the output
url_df_html = url_df.set_index('url')

# Generate HTML with clickable links
html_content = url_df_html.to_html(escape=False, render_links=True)

# Write the HTML content to a file
filename = path_join(config.RESULTS_DATA_PATH, "annotation", f"annotated_run_urls.html")
with open(filename, 'w') as f:
    f.write(html_content)

print(f"HTML file with clickable links has been saved to: {filename}")

HTML file with clickable links has been saved to: /Users/alexandergoodell/code/llm-calc/data/results/annotation/annotated_run_urls.html


## Combine and export
__Combine queued runs with annotated runs to get all runs used for error classifcation__

In [25]:
queued_runs["annotation_status"] = "queued"
revised_annotated_df["annotation_status"] = "annotated"
revised_annotated_df["assigned_to"] = "Alex Goodell"

# Todo: remove
original_10_annotations = ['562d9814-2fe9-4d18-b796-b8a66044b86b',
'562d9814-2fe9-4d18-b796-b8a66044b86b',
'1bac8ca7-ab98-442c-8d5c-88c6b73e8646',
'1bac8ca7-ab98-442c-8d5c-88c6b73e8646',
'0224abf7-f5f3-4e68-93c1-118406f01fcb',
'0224abf7-f5f3-4e68-93c1-118406f01fcb',
'd0eeb840-0624-4dc2-8035-67bd4173ac20',
'd0eeb840-0624-4dc2-8035-67bd4173ac20',
'a61da5a9-290a-47dc-a631-96c01e9e7b7a',
'a61da5a9-290a-47dc-a631-96c01e9e7b7a',
'a8124cf1-97e8-4b62-b276-ca2a002002c1',
'a8124cf1-97e8-4b62-b276-ca2a002002c1',
'4c310462-b675-45e2-a2c8-aa0e6af5db12',
'4c310462-b675-45e2-a2c8-aa0e6af5db12',
'c5c74d69-3173-461b-a592-7e223632c7f5',
'c5c74d69-3173-461b-a592-7e223632c7f5',
'3d8e1215-e86a-4be0-b45c-996f05bbfb52',
'3d8e1215-e86a-4be0-b45c-996f05bbfb52',
'e1a11d07-db27-4ac0-978e-e045c4496162']

for i in range(0, len(original_10_annotations)):
    revised_annotated_df.loc[revised_annotated_df.id == original_10_annotations[i], "assigned_to"] = "Larry Chu"


In [26]:
all_error_class_runs = pd.concat([queued_runs , revised_annotated_df])
print(all_error_class_runs.shape)
all_error_class_runs

(410, 36)


,id,name,start_time,run_type,end_time,extra,error,serialized,events,inputs,...,total_cost,prompt_cost,completion_cost,parent_run_ids,trace_id,dotted_order,in_dataset,assigned_to,annotation_status,index
0,211e216d-dd21-4dd8-b482-db14d90f17b4,execute,2024-10-31 23:26:33.631384,chain,2024-10-31 23:26:46.397041,"{'metadata': {'revision_id': '69fd87a-dirty', 'arm': 'llama_omc', ...",None,None,[],{'inputs': {'human_input': 'I would like you assistance with a cas...,...,0.0082604,0.0081368,0.0001236,[],211e216d-dd21-4dd8-b482-db14d90f17b4,20241031T232633631384Z211e216d-dd21-4dd8-b482-db14d90f17b4,False,Alex Goodell,queued,NaN
0,766ad387-4b88-420e-88da-4abb68c12d10,execute,2024-11-01 18:24:34.530915,chain,2024-11-01 18:24:42.984254,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_rag_ci'...",None,None,[],{'inputs': {'human_input': 'I have a patient that I'd like to tell...,...,0.01498,0.01414,0.00084,[],766ad387-4b88-420e-88da-4abb68c12d10,20241101T182434530915Z766ad387-4b88-420e-88da-4abb68c12d10,False,Alex Goodell,queued,NaN
0,394c3306-1811-477c-8e57-27ad199c6d21,execute,2024-11-01 16:22:59.168627,chain,2024-11-01 16:23:09.437920,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_ci', 'e...",None,None,[],{'inputs': {'human_input': 'A 65-year-old female patient is admitt...,...,0.00784,0.006835,0.001005,[],394c3306-1811-477c-8e57-27ad199c6d21,20241101T162259168627Z394c3306-1811-477c-8e57-27ad199c6d21,False,Alex Goodell,queued,NaN
0,db70c639-20db-41b5-85b7-ac88f427295a,execute,2024-10-30 15:55:59.468902,chain,2024-10-30 15:56:15.859321,"{'metadata': {'revision_id': 'd3184dd-dirty', 'arm': 'gpt4_rag', '...",None,None,[],{'inputs': {'human_input': 'I'm seeking your advice on a liver tra...,...,0.017225,0.00722,0.010005,[],db70c639-20db-41b5-85b7-ac88f427295a,20241030T155559468902Zdb70c639-20db-41b5-85b7-ac88f427295a,False,Alex Goodell,queued,NaN
0,e931862e-d54f-444c-8e5e-20946c677a48,execute,2024-11-01 16:20:40.983930,chain,2024-11-01 16:20:45.895698,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_base', ...",None,None,[],{'inputs': {'human_input': 'Please assist with assessing a patient...,...,0.004385,0.00242,0.001965,[],e931862e-d54f-444c-8e5e-20946c677a48,20241101T162040983930Ze931862e-d54f-444c-8e5e-20946c677a48,False,Alex Goodell,queued,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,5b06b0af-ad6b-44e6-9bd0-5c695e77d8cf,execute,2024-11-01 16:20:59.492591,chain,2024-11-01 16:21:02.098944,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_base', ...",None,None,[],{'inputs': {'human_input': 'Please assist with assessing a patient...,...,0.004375,0.00238,0.001995,[],5b06b0af-ad6b-44e6-9bd0-5c695e77d8cf,20241101T162059492591Z5b06b0af-ad6b-44e6-9bd0-5c695e77d8cf,False,Alex Goodell,annotated,3.0
45,7342591a-54bc-4eea-8353-6597f44975fd,execute,2024-11-01 16:20:45.923236,chain,2024-11-01 16:20:53.559268,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'gpt4_base', ...",None,None,[],{'inputs': {'human_input': 'We have a 72-year-old patient with sep...,...,0.00847,0.00223,0.00624,[],7342591a-54bc-4eea-8353-6597f44975fd,20241101T162045923236Z7342591a-54bc-4eea-8353-6597f44975fd,False,Alex Goodell,annotated,4.0
46,7868a154-8eee-450c-9c84-66179c41874d,execute,2024-11-01 15:17:23.487826,chain,2024-11-01 15:17:31.407414,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'llama_rag', ...",None,None,[],{'inputs': {'human_input': 'A patient with an unknown age presents...,...,0.0011828,0.0011068,0.000076,[],7868a154-8eee-450c-9c84-66179c41874d,20241101T151723487826Z7868a154-8eee-450c-9c84-66179c41874d,False,Alex Goodell,annotated,0.0
47,9f6c7566-4c66-406b-bef2-38919e8fb5a7,execute,2024-11-01 15:16:51.528283,chain,2024-11-01 15:17:01.707505,"{'metadata': {'revision_id': 'a1cd5ba-dirty', 'arm': 'llama_rag', ...",None,None,[],{'inputs': {'human_input': 'I'd like your help with a calculation ...,...,0.00091,0.0008336,0.0000764,[],9f6c7566-4c66-406b-bef2-38919

In [27]:
# add column named hash with random 15 alphanumeric characters with 
# from llm_calc.util import util
# all_error_class_runs["hash"] = all_error_class_runs.id.apply(lambda x: "-".join(util.generate_random_string() for i in range(4)))
# all_error_class_runs.to_pickle(filename)

In [28]:
filename = path_join(config.RESULTS_DATA_PATH, "annotation", f"all_error_class_runs.pkl")
all_error_class_runs.to_pickle(filename)

In [29]:
all_error_class_runs.id.value_counts()

id
211e216d-dd21-4dd8-b482-db14d90f17b4    2
43c56422-9847-43ae-9cd4-aab70896a9ee    2
e9992986-ad5e-41ba-ae31-73a0d7520fb1    2
239c6baa-1343-400e-bfad-60b61bf2bb3e    1
517b06a2-dce4-4b8a-a6b8-dfca6263775b    1
                                       ..
6cc6e863-a194-4c8e-8e07-4d8479cda9f6    1
d467c709-4736-43af-a688-d01b600bcf48    1
f2063684-b176-4401-b935-149e324485e0    1
443ecb48-2790-4e5d-899d-8010ab785786    1
e1a11d07-db27-4ac0-978e-e045c4496162    1
Name: count, Length: 407, dtype: int64

###  Get feedback for all runs (just for show)

In [30]:
feedback = ls_client.list_feedback(feedback_key=["error classification"])
feedback_df = make_df(feedback)
feedback_df

,id,created_at,modified_at,run_id,key,score,value,comment,correction,feedback_source,session_id,comparative_experiment_id,feedback_group_id
0,7ffc262e-d3b5-4da5-ad5b-4fc464c17657,2024-11-15 16:36:05.096985,2024-11-15 16:36:05.411644,3742ed5d-3b59-47a2-b43f-942158de5b41,error classification,0.0,Interpretation Error,,None,type='app' metadata=None,e4246dca-2aa3-48d4-8784-8f73d220d35d,None,None
1,2a354167-229f-4e31-bad1-cd04255da57c,2024-11-15 16:22:01.556357,2024-11-15 16:22:01.859141,211e216d-dd21-4dd8-b482-db14d90f17b4,error classification,7.0,I cannot tell,,None,type='app' metadata=None,4f855ce3-c717-4808-8c40-2a31168ddc3c,None,None
2,5dfc4af3-e555-4c7a-8be4-5ae9c2552e0c,2024-11-15 16:21:05.240129,2024-11-15 16:21:06.033281,c7ed22cb-a087-4f50-b640-db0e81eb274f,error classification,2.0,Assignment Error,,None,type='app' metadata=None,37c33c30-f479-4bd1-a61c-924470f53d35,None,None
3,84406f54-44e3-41f6-bd85-3f82b328c3bc,2024-11-15 16:05:10.484361,2024-11-15 16:05:10.857286,334576b8-f66f-4cc7-b42a-907c073c57df,error classification,0.0,Interpretation Error,,None,type='app' metadata=None,fc384fa5-c928-4745-8cce-404361fabb14,None,None
4,337ecb9a-fe8b-4902-beb2-b583eec9bcf6,2024-11-15 16:03:00.975825,2024-11-15 16:03:01.134845,7868a154-8eee-450c-9c84-66179c41874d,error classification,0.0,Interpretation Error,,None,type='app' metadata=None,01d896de-1de1-4d9d-b1c9-5d30e0cdfdf3,None,None
5,df9011a5-35b9-4141-9e4c-59d7f9a32cc2,2024-11-15 15:28:56.055474,2024-11-15 15:28:56.804174,5b06b0af-ad6b-44e6-9bd0-5c695e77d8cf,error classification,1.0,Incorrect Criteria,,None,type='app' metadata=None,30ea1f63-74d6-4825-9349-e7899c95979a,None,None
6,5fcda3cb-9718-466c-bcf2-5867dd3ec586,2024-11-15 15:26:20.148476,2024-11-15 15:26:20.549358,20e4306d-9f9e-4fbb-9c64-89910581f2d9,error classification,6.0,Software Error/Other,,None,type='app' metadata=None,fedf2ec0-e695-4d61-86b9-e78a3acfc718,None,None
7,ba215d0c-cc49-4576-abc0-43f192a171b1,2024-11-15 15:22:58.539172,2024-11-15 15:23:05.531508,8064b1a2-3732-4536-9718-b5657cb40c20,error classification,7.0,I cannot tell,,None,type='app' metadata=None,9873315c-ab37-4d61-9275-205f4b3a68f4,None,None
8,3e7f93f3-5fe2-431f-944a-f9f941ad16ce,2024-11-15 15:19:01.484808,2024-11-15 15:19:01.887067,0fafaa3b-4e3c-461e-8149-f13f8ab788b0,error classification,2.0,Assignment Error,,None,type='app' metadata=None,87a9983e-9b6d-4533-8ec7-763d4201f390,None,None
9,511f3318-4812-4d32-8988-480e96170cde,2024-11-15 15:09:18.057679,2024-11-15 15:09:18.398848,11b68b56-ebd7-41b6-ac18-a28b511d52f4,error classification,0.0,Interpretation Error,,None,type='app' metadata=None,fedf2ec0-e695-4d61-86b9-e78a3acfc718,None,None
